In [1]:
import requests, time
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from collections import Counter
import functools
import operator
import string
from sklearn.model_selection import train_test_split

In [1]:
def pipe(obj, *fns):
    return reduce(lambda x, y: y(x), [obj] + list(fns))

In [2]:
def feature_target_extraction_inhouse(file, feature_col):
    df_final = pd.read_csv(file)
    df_final.drop(columns = feature_col, inplace = True)
    df_new_final = df_final.fillna('0')
    features = df_new_final[[col for col in df_new_final.columns if col != 'category']]
    features = features.astype(int)
    target = df_new_final[['category']]

    for i in range(target.shape[0]):
        target['category'] = [x.replace(x, '1') if x == 'Deadbeats' else x.replace(x,'0') for x in target['category']]
    
    target = target.astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(
        features, # Features (X)
        target, # Target (y)
        test_size = .2,
        random_state = 42
    )
    return X_train, X_test, y_train, y_test

In [3]:
def feature_target_extraction_tf(file, feature_col):
    df_final = pd.read_csv(file)
    df_final.drop(columns = feature_col, inplace = True)
    df_new_final = df_final.fillna('0')
    
    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf = TfidfVectorizer()
    
    for col in df_new_final.columns:
        
        word_count_vectors = tfidf.fit_transform(df_new_final['case'].values).todense().tolist()

#     word_count_vectors[0][:5]
    features = pd.DataFrame(
        data = word_count_vectors,
        columns = tfidf.get_feature_names()
    )
    
    features = features.astype(int)
    
    target = df_new_final[['category']]

    for i in range(target.shape[0]):
        target['category'] = [x.replace(x, '1') if x == 'Deadbeats' else x.replace(x,'0') for x in target['category']]
    
    target = target.astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(
        features, # Features (X)
        target, # Target (y)
        test_size = .2,
        random_state = 42
    )
    return X_train, X_test, y_train, y_test

In [ ]:
def model_fitting(x):
    